In [1]:
print("hello")

hello


In [2]:
%pip install google-generativeai

  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl.metadata (592 bytes)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
  Using cached grpcio_status-1.73.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached grpcio_status-1.72.1-py3-none-any.whl.metadata (1.1 kB)
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
  Using cached uritemplate-4.2.0-py3-none-any.whl.metadata (2.6 kB)
Using cached google_generativeai-0.8.5-py3-none-any.whl (155 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
Using cached proto_plus-1.26.1-py3-no

  You can safely remove it manually.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

# --- Step 1: Imports & Setup ---
import os
import pandas as pd
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions
import google.generativeai as genai

# Load environment
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)


In [3]:
# Model settings
VECTOR_DB_PATH = "C:/Users/shiva/OneDrive/Documents/Material/Amazon_Ecommorce_agent/data/vectorstore/"
COLLECTION_NAME = "products"
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

print("✅ Environment ready")

✅ Environment ready


In [4]:
# --- Step 2: Connect to Chroma Vectorstore ---
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBED_MODEL
)

client_chroma = chromadb.PersistentClient(path=VECTOR_DB_PATH)
collection = client_chroma.get_or_create_collection(
    name=COLLECTION_NAME
    
)

print("✅ Connected to Chroma vectorstore")


✅ Connected to Chroma vectorstore


In [5]:
# --- Step 3: Retrieval Helper ---
def retrieve_products(query, n_results=5):
    """Retrieve top products for a query."""
    results = collection.query(query_texts=[query], n_results=n_results)
    df = pd.json_normalize(results["metadatas"][0])
    df["similarity"] = results["distances"][0]
    df = df.sort_values("similarity")
    return df, results

In [6]:
def get_gemini_model():
    """Initialize Gemini Pro model (free tier)."""
    model = genai.GenerativeModel('gemini-1.5-flash')  # Fast & free
    print("🤖 Gemini model loaded")
    return model

llm = get_gemini_model()

🤖 Gemini model loaded


In [7]:
# --- Step 5: RAG Response Generator ---
def generate_rag_response(query, top_k=5):
    """Combine retrieval and generation using Gemini."""
    retrieved_df, results = retrieve_products(query, n_results=top_k)
    
    # Build concise context
    context = "\n".join([
        f"• {item.get('name','N/A')} — ₹{item.get('discount_price','N/A')} | ⭐ {item.get('ratings','N/A')} ({item.get('no_of_ratings','0')} reviews) | Category: {item.get('sub_category','')}"
        for item in results["metadatas"][0]
    ])
    
    prompt = f"""You are a helpful e-commerce assistant for an Indian online store.

User Query: "{query}"

Available Products:
{context}

Task: Recommend the best 2-3 products from the list above. Explain:
- Why they match the user's needs
- Key features and value proposition
- Price comparison if relevant

Keep your response friendly, concise (3-4 sentences), and use Indian Rupee format (₹).
"""
    
    # Generate with Gemini
    response = llm.generate_content(prompt)
    return retrieved_df, response.text


In [8]:
# --- Step 6: Test Query ---
query = "Best budget smartphone under ₹15000 with good camera"

retrieved_df, answer = generate_rag_response(query)

print("📦 Retrieved Products:")
print(retrieved_df[["name", "discount_price", "ratings", "sub_category"]].head().to_string(index=False))

print("\n🤖 AI Assistant Response:\n")
print(answer)

NotFound: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

In [ ]:
# --- Step 7: Try More Queries ---
test_queries = [
    "Best gaming laptop under ₹70000 with RTX graphics",
    "Wireless earbuds with long battery life under ₹2000",
    "Smartwatch with AMOLED display under ₹5000",
    "Recommend a good air conditioner for a small bedroom"
]

print("\n" + "="*80)
print("🧪 Testing Multiple Queries")
print("="*80)

for q in test_queries:
    print(f"\n🔍 Query: {q}")
    print("-" * 80)
    _, response = generate_rag_response(q)
    print(response)
    print()